## Homework 2: 
## Discovery of Frequent Itemsets and Association Rules
### Group 11

Boyu Xue (boyux@kth.se), Kristupas Bajarunas(kristupasbajarunas@gmail.com)


### Description:

The problem of discovering association rules between itemsets in a sales transaction database (a set of baskets) includes the following two sub-problems [R. Agrawal and R. Srikant, VLDB '94]:

#### Finding frequent itemsets with support at least s;
Generating association rules with confidence at least c from the itemsets found in the first step.
Remind that an association rule is an implication X → Y, where X and Y are itemsets such that X∩Y=∅. Support of the rule X → Y is the number of transactions that contain X⋃Y. Confidence of the rule X → Y the fraction of transactions containing X⋃Y in all transactions that contain X.

#### Task
You are to solve the first sub-problem: to implement the Apriori algorithm for finding frequent itemsets with support at least s in a dataset of sales transactions. Remind that support of an itemset is the number of transactions containing the itemset. To test and evaluate your implementation, write a program that uses your Apriori algorithm implementation to discover frequent itemsets with support at least s in a given dataset of sales transactions.

The implementation can be done using any big data processing framework, such as Apache Spark, Apache Flink, or no framework, e.g., in Java, Python, etc.  

#### Optional task for extra bonus

Solve the second sub-problem, i.e., develop and implement an algorithm for generating association rules between frequent itemsets discovered by using the Apriori algorithm in a dataset of sales transactions. The rules must have support at least s and confidence at least c, where s and c are given as input parameters.

#### 1. Import libiaries and tools

In [1]:
import numpy as np
from collections import defaultdict
import itertools
import random
import time

#### 2. Read the file

1. Split the space " " between the transactions from the file.
2. Return the singletons as integers and append them to the database.

In [2]:
data_file="./data/T10I4D100K.dat"
database = []
for l in open(data_file, 'r'):
    items = l.strip().split(' ')
    singletons = list(map(int, items)) # convert items to integers
    database.append(singletons)

#### 3. Creat some parameters

1. Support threshold of frequent itemsets:Since there are many items(100,000) in this database, the support degree of each frequent itemsets will be very low. So we use 1% as a support degree, which given a support threshould of 1,000.

2. "max_k" is the maximum number of items in a candidate k-itemsets, we use 5 as the value of the "max_k".

3. For "k" in the range of "max_k", we create a blank dictionary "L" which will contains all the frequent itemsets.


In [3]:
support=100000*0.01
conf=0.5
max_k=5
L=[None] * max_k

#### 4. Define the function "createC1" to get the candidate singletons

1. C1 is the candidate singletons sets that might be frequent sets.
2. We use "frozenset" to create the keys in the dictionary of frequent sets. 

In [4]:
def createC1(database):
    C1 = defaultdict(set)
    transaction_num=-1
    for transaction in database:
        transaction_num+=1
        for item in transaction:
            C1[frozenset({item})].add(transaction_num)
    return C1


In [5]:
C1 = createC1(database)

#### 5. Define the "filter_remove" function to filter and remove the non-frequent items

1. Create a delete_list which include the non-frequent items according to the support threshold.
2. Delete the non-frequent items from the candidate singleton list C1.
3. Put the set of truly frequent singletons on dictionary L, as the first set L[0].

In [6]:
def filter_remove(C1,support):
    delete_list=[]
    for key in C1.keys():
        if len(C1[key])<support:
            delete_list.append(key)
    for item in delete_list:
        del(C1[item])
    return C1

In [7]:
L[0]=filter_remove(C1,support)
#L[0]

#### 6. Filter and create the set of truly frequent k-itemsets
1. Combine the L[0] with L[k-2]. In the first loop, we can create a new 2-itemset which include two singletons, and return the truly frequent pairs; in the second loop, we can create a triple which include one singleton and one pair, and return the truly frequent triples.
2. For each new k-itemset, get the values of (k-1)-itemset and 1-itemset, and calculate the intersection of the two values. Then we can get the new value for the new k-itemset, e.g., if we use “frozenset({25}) :{0,2}” and “frozenset({52}) :{0,3}”  to create a new 2-itemset “frozenset({25, 52}” , the value of  “frozenset({25, 52})” will be the intersection of {0,2} and {0,3} ={0}, then we get “frozenset({25, 52}: {0}” .
2. Print the result of frequent k_itemsets (singletons, pairs, triples).  Enumerate some of the result (no more than 50 in each k_itensets). 

In [8]:
for k in range(2, max_k+1):
    single_prod = L[0]
    previous_prod = L[k - 2]
    new_prod_set = defaultdict(set)
    for keyA in single_prod.keys():
        for keyB in previous_prod.keys():
            new_item_set = frozenset(keyA.union(keyB))
            if len(new_item_set) != k:
                continue
            new_prod_set[new_item_set] = single_prod[keyA].intersection(previous_prod[keyB])
    filter_remove(new_prod_set, support)
    L[k-1] = new_prod_set

In [9]:
for k, k_itemsets in enumerate(L,1):
    print("Number of frequent {}-itemsets with support {} = {}".format(k, support, len(k_itemsets)))
    for count, (itemset, transactions) in enumerate(k_itemsets.items(),1):
        print("Items: {} -> Support: {}".format(itemset, len(transactions)))
        if count == 10:
              break

Number of frequent 1-itemsets with support 1000.0 = 375
Items: frozenset({25}) -> Support: 1395
Items: frozenset({52}) -> Support: 1983
Items: frozenset({240}) -> Support: 1399
Items: frozenset({274}) -> Support: 2628
Items: frozenset({368}) -> Support: 7828
Items: frozenset({448}) -> Support: 1370
Items: frozenset({538}) -> Support: 3982
Items: frozenset({561}) -> Support: 2783
Items: frozenset({630}) -> Support: 1523
Items: frozenset({687}) -> Support: 1762
Number of frequent 2-itemsets with support 1000.0 = 9
Items: frozenset({368, 682}) -> Support: 1193
Items: frozenset({368, 829}) -> Support: 1194
Items: frozenset({825, 39}) -> Support: 1187
Items: frozenset({704, 825}) -> Support: 1102
Items: frozenset({704, 39}) -> Support: 1107
Items: frozenset({227, 390}) -> Support: 1049
Items: frozenset({722, 390}) -> Support: 1042
Items: frozenset({217, 346}) -> Support: 1336
Items: frozenset({829, 789}) -> Support: 1194
Number of frequent 3-itemsets with support 1000.0 = 1
Items: frozenset

#### 7.Generate association rules and calculate the confidence of rules

1. The association rules I -> j means 2 or more than 2 items( I(i1,i2,...Ik) and j) usually been included in the same basket, so we only take the frequent 2-itemsets and 3-itemsets (L[1:]) to the following analysis.
2. Generate new set (genset) includes x length of elements from the input frequent itemsets. For example, for {25,52}, if x=0, the new "genset" include 1 item from the itemsets, it will be [(25,), (52,)], if x=1, the genset will be [(25, 52)].
3. Generate "newset" which is used to be the associated item "I" of the association rules, and "frozen" the newset to create the keys in a dictionary. 
4. Generate the "denom" as the associated item "j" of the association rules. The length of denom = 0 means there is no associated item "j".
5. The "numer_support" is the support of itemsets which contain both "I" and "j", the "newset_support" is the support of itemsets which contain the the associated item "I" of the association rules. The confidence of a rule can be calculated by using numer_support/newset_support. (Note: in L[0] each itemset include one frequent item, which means the length of newset=1.)
6. Return the association rules according to a specific confident threshold("conf"). 

In [10]:
def rules(L,conf):
    rules=[]
    for val in L[1:]:
        for items, trans in val.items():
            numer_support=len(trans)
            for x in range(len(items)):
                genset=list(itertools.combinations(items, x+1))
                for sets in genset:
                    newset=frozenset(sets)
                    newset_support=len(L[len(newset)-1][newset])# len(pairs)=2, L[1] is about the pairs.
                    denom=items-newset
                    if len(denom)==0:
                        continue
                    confidence=numer_support/newset_support
                    if confidence>=conf:
                        rules.append((newset,denom,confidence))
    return rules

In [11]:
r = rules(L, conf)
n_rules = len(r)
for i in range(n_rules):
    m1 = list(r[i][0])[0:]
    m2=list(r[i][1])[0:]
    m3=r[i][2]
    print("{} -> {!s:<30}(confidence: {})".format(m1,m2,m3))

[704] -> [825]                         (confidence: 0.6142697881828316)
[704] -> [39]                          (confidence: 0.617056856187291)
[227] -> [390]                         (confidence: 0.577007700770077)
[704] -> [825, 39]                     (confidence: 0.5769230769230769)
[704, 825] -> [39]                          (confidence: 0.9392014519056261)
[704, 39] -> [825]                         (confidence: 0.9349593495934959)
[825, 39] -> [704]                         (confidence: 0.8719460825610783)
